In [70]:
import os
import numpy as np
import random
import glob
import pandas as pd
from tqdm import tqdm_notebook
from pyquaternion import Quaternion
from lyft_dataset_sdk.lyftdataset import LyftDataset
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud, Box
from lyft_dataset_sdk.utils.geometry_utils import BoxVisibility, transform_matrix
from lyft_dataset_sdk.utils.kitti import KittiDB
from lyft_dataset_sdk.eval.detection.mAP_evaluation import Box3D

In [2]:
# load lyft test dataset
level5data = LyftDataset(data_path='.', json_path='/home/bob/data/lyft_data/test_data', verbose=True)

JSON file instance.json missing, using empty list
JSON file sample_annotation.json missing, using empty list
9 category,
17 attribute,
4 visibility,
0 instance,
8 sensor,
168 calibrated_sensor,
219744 ego_pose,
218 log,
218 scene,
27468 sample,
219744 sample_data,
0 sample_annotation,
1 map,
Done loading in 3.4 seconds.
Reverse indexing ...
Done reverse indexing in 1.5 seconds.


In [71]:
# loda kitti pred boxes
pred_folder = '/home/bob/data/lyft2kitti/testing/pred/'
pred_filepaths = sorted(glob.glob(os.path.join(pred_folder, "*.txt")))

In [72]:
def read_calib_file(filepath):
        with open(filepath) as f:
            lines = f.readlines()
    
        obj = lines[2].strip().split(' ')[1:]
        P2 = np.array(obj, dtype=np.float32)
        obj = lines[3].strip().split(' ')[1:]
        P3 = np.array(obj, dtype=np.float32)
        obj = lines[4].strip().split(' ')[1:]
        R0 = np.array(obj, dtype=np.float32)
        obj = lines[5].strip().split(' ')[1:]
        Tr_velo_to_cam = np.array(obj, dtype=np.float32)

        return {'P2': P2.reshape(3, 4),
                'P3': P3.reshape(3, 4),
                'R_rect': R0.reshape(3, 3),
                'Tr_velo2cam': Tr_velo_to_cam.reshape(3, 4)}

In [74]:
pred_box3ds = []
kitti_to_nu_lidar = Quaternion(axis=(0,0,1), angle=np.pi/2)
for ii, pred_file in enumerate(pred_filepaths):
    sample_token = pred_file[40:-4]
    calib_path = pred_file.replace('pred','calib')
    calib = read_calib_file(calib_path)
    sample = level5data.get('sample', sample_token)
    lidar_token = sample['data']['LIDAR_TOP']
    sd_record_lid = level5data.get('sample_data', lidar_token) # lidar sample data
    cs_record_lid = level5data.get('calibrated_sensor', sd_record_lid['calibrated_sensor_token']) 
    ego_pose = level5data.get("ego_pose", sd_record_lid["ego_pose_token"])
    with open(pred_file) as f:
        for line in f:
            # Parse this line into box information.
            parsed_line = KittiDB.parse_label_line(line) # in cam corrd
            center = parsed_line["xyz_camera"]
            wlh = parsed_line["wlh"]
            yaw_camera = parsed_line["yaw_camera"]
            name = parsed_line["name"]
            score = parsed_line["score"]
            # quat_box 
            quat_box = Quaternion(axis=(0, 1, 0), angle=yaw_camera) * Quaternion(axis=(1, 0, 0), angle=np.pi / 2)
            # 1: box in camera coord
            box = Box([0.0, 0.0, 0.0], wlh, quat_box, name=name, token = sample_token)
            # 2: center definition difference
            box.translate(center + np.array([0, -wlh[2] / 2, 0]))
            # 3: transform from camera to lidar
            box.rotate(Quaternion(matrix=calib['R_rect']).inverse)
            box.translate(-calib['Tr_velo2cam'][:,3])
            box.rotate(Quaternion(matrix=calib['Tr_velo2cam'][:,:3]).inverse)
            # 4: Transform to nuScenes LIDAR coord system.
            box.rotate(kitti_to_nu_lidar)
            box.score = score
            # 5: transform from lidar to ego
            box.rotate(Quaternion(cs_record_lid['rotation']))
            box.translate(np.array(cs_record_lid['translation']))
            # 6: transform from ego to global
            box.rotate(Quaternion(ego_pose['rotation']))
            box.translate(np.array(ego_pose['translation']))
            
            ## to 3D box format
            box3d = Box3D(
                sample_token=sample_token,
                translation=list(box.center),
                size=list(box.wlh),
                rotation=list(box.orientation.elements),
                name=name,
                score=score
            )
        pred_box3ds.append(box3d)

In [ ]:
sub = {}
for i in tqdm_notebook(range(len(pred_box3ds))):
#     yaw = -np.arctan2(pred_box3ds[i].rotation[2], pred_box3ds[i].rotation[0])
    yaw = 2*np.arccos(pred_box3ds[i].rotation[0])
    pred =  str(pred_box3ds[i].score/255) + ' ' + str(pred_box3ds[i].center_x)  + ' '  + \
    str(pred_box3ds[i].center_y) + ' '  + str(pred_box3ds[i].center_z) + ' '  + \
    str(pred_box3ds[i].width) + ' ' \
    + str(pred_box3ds[i].length) + ' '  + str(pred_box3ds[i].height) + ' ' + str(yaw) + ' ' \
    + str(pred_box3ds[i].name) + ' ' 
        
    if pred_box3ds[i].sample_token in sub.keys():     
        sub[pred_box3ds[i].sample_token] += pred
    else:
        sub[pred_box3ds[i].sample_token] = pred        
    
sample_sub = pd.read_csv('/home/bob/data/lyft_data/sample_submission.csv')
for token in set(sample_sub.Id.values).difference(sub.keys()):
    print(token)
    sub[token] = ''

sub = {}           
for i in tqdm_notebook(range(len(pred_boxes))):
    box = pred_boxes[i]
    yaw = 2*np.arccos(box.orientation[0])
    pred =  str(box.score) + ' ' + str(box.center[0])  + ' '  + str(box.center[1]) + ' '  + \
        str(box.center[2]) + ' '  + str(box.wlh[0]) + ' '+ str(box.wlh[1]) + ' '  + str(box.wlh[2])\
        + ' ' + str(yaw) + ' ' + str(box.name) + ' ' 
        
    if box.token in sub.keys():     
        sub[box.token] += pred
    else:
        sub[box.token] = pred        
    
sample_sub = pd.read_csv('/home/bob/data/lyft_data/sample_submission.csv')
count = 0
for token in set(sample_sub.Id.values).difference(sub.keys()):
    print(token)
    count+=1
    sub[token] = ''
print(count)

In [76]:
sub = pd.DataFrame(list(sub.items()))
sub.columns = sample_sub.columns
sub.head()

,Id,PredictionString
0,0005e1fcd0f431ccfc5d301c193da1d1fcd6007c104566...,1.0 2424.2783973882256 844.2727629191078 -20.4...
1,0006881ddeae36eec1d4f5d3b3c0d4cf23a5afdd668833...,1.0 631.9738700169175 2630.1338843974518 -21.8...
2,000e0f4e29ef684c0a2ccf9bd17586e9b2491e9f488595...,1.0 1988.6995395590056 1140.2357526255125 -20....
3,0013a4dccf084f18587ad99a20f7c7bc32dc23e10dcd36...,1.0 1165.8717303351834 1642.0893989460772 -25....
4,0015c9d8d97b2409c6283f7dfa9fdfd1f016ed3146e34d...,1.0 1133.5052863119015 1626.865814049319 -25.2...


In [77]:
sub.to_csv('complex_yolo_test.csv', index=False)
sub.shape

(27468, 2)

In [81]:
sub['0bb5c8961726da83a833206bdc086a71e3f6e6288b46319a714ada68b32e263b']

''

In [62]:
sub.iloc[0]

Id                  0005e1fcd0f431ccfc5d301c193da1d1fcd6007c104566...
PredictionString    -1.0 2424.2783973882256 844.2727629191078 -20....
Name: 0, dtype: object